In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from plasmistry.electron import EEDF
from plasmistry.electron import get_maxwell_eedf
from plasmistry.molecule import get_ideal_gas_density

In [2]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,9)
plt.rcParams["font.size"] = 14

In [3]:
np.set_printoptions(precision=1, linewidth=130, edgeitems=5)
pd.options.display.float_format = '{:,.1e}'.format

### Read inelas reactions

In [4]:
inelas_df_forward = pd.read_pickle('rctn_dataframe_forward.pkl')
inelas_df_backward = pd.read_pickle('rctn_dataframe_backward.pkl')

In [5]:
inelas_df_forward.loc[0, 'cross_section']

array([[1.0e-04, 1.0e-04, 1.0e-04, 1.0e-04, 1.0e-04, ..., 9.9e+02, 1.0e+03, 1.0e+03, 1.0e+03, 1.0e+03],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, ..., 2.2e-25, 2.2e-25, 2.2e-25, 2.2e-25, 2.2e-25]])

In [6]:
inelas_df = pd.concat([inelas_df_forward, inelas_df_backward], ignore_index=True)
# inelas_df = pd.read_pickle("rctn_dataframe_H2_dis_via_b.pkl")
inelas_df = pd.read_pickle("CO2_ele_vib_forward_df.pkl")

In [7]:
inelas_df

,formula,type,threshold_eV,cross_section
0,E + CO2 => E + CO2(v1),excitation,2.9e-01,"[[0.291, 0.3, 0.31, 0.32, 0.33, 0.35, 0.38, 0...."
1,E + CO2 => E + CO2(v2),excitation,5.8e-01,"[[0.57939, 0.58839, 0.59839, 0.60839, 0.61839,..."
2,E + CO2 => E + CO2(v3),excitation,8.6e-01,"[[0.86469, 0.87369, 0.88369, 0.89369, 0.90369,..."
3,E + CO2 => E + CO2(v4),excitation,1.1e+00,"[[1.1469, 1.1559, 1.1659, 1.1759, 1.1859, 1.20..."
4,E + CO2 => E + CO2(v5),excitation,1.4e+00,"[[1.42601, 1.43501, 1.44501, 1.45501, 1.46501,..."
5,E + CO2 => E + CO2(v6),excitation,1.7e+00,"[[1.70203, 1.71103, 1.72103, 1.73103, 1.74103,..."
6,E + CO2 => E + CO2(v7),excitation,2.0e+00,"[[1.97495, 1.98395, 1.99395, 2.00395, 2.01395,..."
7,E + CO2 => E + CO2(v8),excitation,2.2e+00,"[[2.24478, 2.25378, 2.26378, 2.27378, 2.28378,..."
8,E + CO2 => E + CO2(v9),excitation,2.5e+00,"[[2.51151, 2.52051, 2.53051, 2.54051, 2.55051,..."
9,E + CO2 => E + CO2(v10),excitation,2.8e+00,"[[2.77515, 2.78415, 2.79415, 2.80415, 2.81415,..."


In [8]:
inelas_df.loc[0, 'cross_section']

array([[2.9e-01, 3.0e-01, 3.1e-01, 3.2e-01, 3.3e-01, 3.5e-01, 3.8e-01, 4.0e-01, 4.5e-01, 5.0e-01, 6.0e-01, 8.0e-01, 1.0e+00,
        1.5e+00, 2.0e+00, 3.0e+00, 4.5e+00, 6.0e+00, 8.0e+00, 1.0e+01, 2.5e+01, 3.0e+01, 1.0e+02],
       [0.0e+00, 9.5e-21, 1.7e-20, 1.9e-20, 2.0e-20, 2.2e-20, 2.2e-20, 2.2e-20, 2.0e-20, 1.9e-20, 1.5e-20, 1.2e-20, 1.0e-20,
        7.6e-21, 6.4e-21, 4.9e-21, 4.4e-21, 4.1e-21, 4.8e-21, 2.6e-21, 1.4e-21, 1.0e-21, 0.0e+00]])

### Choose the reactions considered.

In [9]:
#inelas_df = inelas_df.iloc[[0,1,2,3,4,5,6,7,8]].reset_index(drop=True)
inelas_df

,formula,type,threshold_eV,cross_section
0,E + CO2 => E + CO2(v1),excitation,2.9e-01,"[[0.291, 0.3, 0.31, 0.32, 0.33, 0.35, 0.38, 0...."
1,E + CO2 => E + CO2(v2),excitation,5.8e-01,"[[0.57939, 0.58839, 0.59839, 0.60839, 0.61839,..."
2,E + CO2 => E + CO2(v3),excitation,8.6e-01,"[[0.86469, 0.87369, 0.88369, 0.89369, 0.90369,..."
3,E + CO2 => E + CO2(v4),excitation,1.1e+00,"[[1.1469, 1.1559, 1.1659, 1.1759, 1.1859, 1.20..."
4,E + CO2 => E + CO2(v5),excitation,1.4e+00,"[[1.42601, 1.43501, 1.44501, 1.45501, 1.46501,..."
5,E + CO2 => E + CO2(v6),excitation,1.7e+00,"[[1.70203, 1.71103, 1.72103, 1.73103, 1.74103,..."
6,E + CO2 => E + CO2(v7),excitation,2.0e+00,"[[1.97495, 1.98395, 1.99395, 2.00395, 2.01395,..."
7,E + CO2 => E + CO2(v8),excitation,2.2e+00,"[[2.24478, 2.25378, 2.26378, 2.27378, 2.28378,..."
8,E + CO2 => E + CO2(v9),excitation,2.5e+00,"[[2.51151, 2.52051, 2.53051, 2.54051, 2.55051,..."
9,E + CO2 => E + CO2(v10),excitation,2.8e+00,"[[2.77515, 2.78415, 2.79415, 2.80415, 2.81415,..."


In [10]:
inelas_df.loc[0, 'cross_section']

array([[2.9e-01, 3.0e-01, 3.1e-01, 3.2e-01, 3.3e-01, 3.5e-01, 3.8e-01, 4.0e-01, 4.5e-01, 5.0e-01, 6.0e-01, 8.0e-01, 1.0e+00,
        1.5e+00, 2.0e+00, 3.0e+00, 4.5e+00, 6.0e+00, 8.0e+00, 1.0e+01, 2.5e+01, 3.0e+01, 1.0e+02],
       [0.0e+00, 9.5e-21, 1.7e-20, 1.9e-20, 2.0e-20, 2.2e-20, 2.2e-20, 2.2e-20, 2.0e-20, 1.9e-20, 1.5e-20, 1.2e-20, 1.0e-20,
        7.6e-21, 6.4e-21, 4.9e-21, 4.4e-21, 4.1e-21, 4.8e-21, 2.6e-21, 1.4e-21, 1.0e-21, 0.0e+00]])

In [11]:
eedf = EEDF(max_energy_eV=50, grid_number=500)

In [12]:
eedf.initialize(rctn_with_crostn_df=inelas_df, total_species=['E', 'H2'] + [f'H2(v{v})' for v in range(1, 15)])
# eedf.initialize(rctn_with_crostn_df=inelas_df, total_species=['E', 'H2', 'H2(v1)', 'H2(v2)', 'H2(v3)'])

AssertionError: 

In [ ]:
eedf.rate_const_matrix_e_inelas_electron.toarray()

In [ ]:
eedf.rate_const_matrix_e_inelas_electron.shape

In [ ]:
eedf.rate_const_matrix_e_inelas_molecule.shape

In [ ]:
plt.matshow(eedf.rate_const_matrix_e_inelas_molecule)

In [ ]:
eedf.n_bg_molecule_elas

In [ ]:
eedf.bg_molecule_elas

In [ ]:
eedf.n_bg_molecule_inelas

In [ ]:
eedf.bg_molecule_inelas

In [ ]:
eedf.n_inelas_rctn

In [ ]:
eedf.bg_molecule_inelas_in_order

In [ ]:
plt.spy(eedf.rate_const_matrix_e_inelas_electron, markersize=2)

In [ ]:
eedf.rate_const_matrix_e_inelas_molecule

In [ ]:
N = get_ideal_gas_density(p_Pa=1e5, Tgas_K=1000)
N

### Set parameters

In [ ]:
eedf.set_parameters(E=2e5, Tgas=1000, N=N)

In [ ]:
total_species_density = np.ones(16) * N
# total_species_density = np.array([0,1,0,0,0])*N
def dndt_all(t, y):
    eedf.set_density_per_J(y)
    eedf.set_flux(total_species_density=total_species_density)
    return eedf.get_deriv_total(total_species_density=total_species_density) 

In [ ]:
y0 = get_maxwell_eedf(eedf.energy_point, Te_eV=0.7)*1e17
time_span = [0, 1e3]
sol = solve_ivp(dndt_all, time_span, y0, method="BDF")

In [ ]:
sol.t

In [ ]:
plt.semilogy(eedf.energy_point_eV, (sol.y.transpose()/np.sqrt(eedf.energy_point)).transpose()[:,::50])

In [ ]:
eedf.set_density_per_J(sol.y[:,0])
eedf.plot_normalized_eepf()

In [ ]:
eedf.plot_normalized_eedf()

In [ ]:
eedf.set_density_per_J(sol.y[:,-1])
eedf.plot_normalized_eepf(xlim=(0,12), ylim=(1e-12, 1e1))

In [ ]:
print(eedf)

In [ ]:
eedf._get_molecule_rate_const_e_inelas()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.barh(range(eedf.n_inelas_rctn),
        width=eedf._get_molecule_rate_const_e_inelas(),
        height=0.6, log=True)
ax.set_yticks(range(eedf.n_inelas_rctn))
ax.set_yticklabels(eedf.inelas_reaction_dataframe['formula'])

In [ ]:
eedf.plot_normalized_eedf()

In [ ]:
Te_seq = []
ne_seq = []
for _array in sol.y.transpose():
    eedf.set_density_per_J(_array)
    Te_seq.append(eedf.electron_temperature_in_eV)
    ne_seq.append(eedf.electron_density)
plt.semilogx(sol.t, Te_seq, marker='.')
plt.title("Te vs. t")

In [ ]:
plt.semilogx(sol.t, ne_seq, marker='.')
plt.title("ne vs. t")

In [ ]:
sol.y

In [ ]:
eedf.set_density_per_J(sol.y[:,-1])
eedf.set_flux(total_species_density=total_species_density)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ef(), 
        linewidth=0.4, marker='.', label='ef')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_el(), 
        linewidth=0.4, marker='.', label='el')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ee(), 
        linewidth=0.4, marker='.', label='ee')
ax.plot(eedf.energy_point_eV, eedf._get_electron_rate_e_inelas(density=total_species_density), 
        linewidth=0.4, marker='.', label='e_inelas')
ax.plot(eedf.energy_point_eV, eedf.get_deriv_total(total_species_density=total_species_density), 
        linewidth=1.5, marker='.', label='sum')
# ax.plot(eedf.energy_nodes,eedf.J_flux_el, marker='.', label='el')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ee, marker='.', label='ee')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ef+eedf.J_flux_el+eedf.J_flux_ee, marker='.', label='ef+el+ee')
# ax.set_yscale('symlog')
# ax.set_xscale('log')
ax.set_xlim((0,20))
ax.legend()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ef()*eedf.energy_point, 
        linewidth=0.4, marker='.', label='ef')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_el()*eedf.energy_point, 
        linewidth=0.4, marker='.', label='el')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ee()*eedf.energy_point, 
        linewidth=0.4, marker='.', label='ee')
ax.plot(eedf.energy_point_eV, eedf._get_electron_rate_e_inelas(density=total_species_density)*eedf.energy_point, 
        linewidth=0.4, marker='.', label='e_inelas')
ax.plot(eedf.energy_point_eV, eedf.get_deriv_total(total_species_density=total_species_density)*eedf.energy_point, 
        linewidth=1.5, marker='.', label='sum')
# ax.plot(eedf.energy_nodes,eedf.J_flux_el, marker='.', label='el')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ee, marker='.', label='ee')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ef+eedf.J_flux_el+eedf.J_flux_ee, marker='.', label='ef+el+ee')
# ax.set_yscale('symlog')
# ax.set_xscale('log')
ax.set_xlim((0,20))
ax.legend()

In [ ]:
energy_gain_ef = (eedf._get_deriv_ef()*eedf.energy_point).sum()
energy_lose_el = (eedf._get_deriv_el()*eedf.energy_point).sum()
energy_gain_ee = (eedf._get_deriv_ee()*eedf.energy_point).sum()
energy_gain_e_inelas = (eedf._get_electron_rate_e_inelas(density=total_species_density)*eedf.energy_point).sum()
energy_gain_net = (eedf.get_deriv_total(total_species_density=total_species_density)*eedf.energy_point).sum()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.barh(range(5),
        width=[energy_gain_ef, energy_lose_el, energy_gain_ee, energy_gain_e_inelas, energy_gain_net],
       height=0.6)
ax.set_yticks(range(5))
ax.set_yticklabels(['ef', 'el', 'ee', 'inelas', 'net'])
ax.set_title("energy lose")

In [ ]:
eedf.inelas_rctn_info